# PageRank - Explicado

## Definiciones  

 - $PR(usuario_i)$: PageRank calculado para la el usuario $i$.
 - Backlink: Si la el usuario $A$ sigue a $B$, entonces se dice que el usuario $B$ tiene un *backlink* desde $A$.
 
## ¿Qué es PageRank?  

En resumen, PageRank es una especie de "voto", otorgado por todos los usuarios de twitter qué tan importante un determinado usuario es. Cada follow cuenta como un "voto".

## ¿Cómo funciona?  

Supongamos que tenemos $N$ usuarios en nuestro universo. Y si el usuario $A$ es seguido por $u_1$, $u_2$, $...$, $u_n$, el PageRank del usuario $A$ está determinado por:  

$$PR(A) = \frac{1 - d}{N} + d \left( \frac{PR(u_1)}{L(u_1)} +  \frac{PR(u_2)}{L(u_2)} +\space...\space+  \frac{PR(u_n)}{L(u_n)} \right)$$

Nuevos términos: 

 -  $L(u_n)$ - El término $L(u_n)$ se refiere el número de usuarios que el usuario $u_n$ sigue, su contribución al PageRank de cualquiera de las personas que sigue se ve "diluido" entre más usuarios siga.
 - $d$ - es conocida como factor de moderación. Un valor común es $0.85$.  

## ¿Iterativo?  

No es suficiente con calcular el PageRank una sola vez, puesto que los valores fluctuan, la importancia de un determinado usuario no puede ser calculado en una sola operación... sin embargo, con el tiempo, los valores de PageRank convergen, es decir, se mantienen estables. La fórmula en realidad se divide en:  

Cuando $t = 0$:  

$$PR(u_i;0) = \frac{1}{N}$$

Para cualquier otro valor de $t$:  

$$PR(u_i;t) = \frac{1 - d}{N} + d \sum_{u_j \in S(u_i) } \frac{PR(u_j;t-1)}{L(u_j)}$$

## Un micro ejemplo  

In [8]:
from graphviz import Digraph

twitter_graph = Digraph("Un ejemplo")

twitter_graph.node("1", "io_exception")

twitter_graph.render()

ExecutableNotFound: failed to execute ['dot', '-Tpdf', '-O', 'Un ejemplo.gv'], make sure the Graphviz executables are on your systems' PATH

In [34]:
followers = defaultdict(set)
following = defaultdict(set)

In [35]:
with open("twitter.jsonl") as r:
    for line in r:
        record = json.loads(line)
        following[record["user_id"]].update(record["following_id"])

with open("twitter.jsonl") as r:
    for line in r:
        record = json.loads(line)
        for user_id in record["following_id"]:
            if user_id in following:
                followers[user_id].add(record["user_id"])

In [36]:
len(followers)

77

In [37]:
len(following)

88

In [38]:
followers

defaultdict(set,
            {174104147: {12741082,
              18766800,
              41852612,
              60729429,
              84474379,
              107245235,
              109409138,
              109736946,
              112907123,
              143183218,
              178848278,
              183064762,
              226437753,
              234086009,
              268088205,
              851390880,
              852306660,
              2469849156,
              2582486138,
              891085040007999490,
              1039473529568329728,
              1121236564271685634},
             3643495637: {12741082,
              39174346,
              60729429,
              84474379,
              112907123,
              148388154,
              178848278,
              268088205,
              1096466461,
              1569333570,
              2582486138,
              891085040007999490,
              936598053850578944,
              1039473529568329728,
      